In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy.misc import comb
from sklearn.model_selection import train_test_split , KFold, StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.preprocessing import normalize
import pdb
from sklearn.metrics import roc_auc_score , make_scorer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import NuestroArbol as ourTree
import random
from scipy.stats import randint as sp_randint
from time import time
import collections
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from IPython.core.display import HTML

%matplotlib inline

# Importo datos 

In [2]:
X_comp = pd.read_csv('X_competencia.csv')
X = pd.read_csv('X.csv')
y = pd.read_csv('y.csv')
X.drop(['index'],inplace=True, axis=1)
y.drop(['index'], inplace=True,axis=1)



FileNotFoundError: File b'X_competencia.csv' does not exist

In [ ]:
#partimos los datos en desarrollo(87%) y holdout(13%). 
X_desarrollo , X_holout ,y_desarrollo, y_holdout = train_test_split(X, y['output'],
                                                                    test_size=0.13,random_state=0,stratify=y['output'])

#a = normalize(X_desarrollo, axis=0, copy=True, return_norm=False)
#from IPython.core.debugger import Tracer; Tracer()() 
#X_desarrollo = pd.DataFrame(SelectKBest(chi2, k=180).fit_transform(np.abs(X_desarrollo), y_desarrollo))

#pase los kfold aca arriba ya que se usan en varios lugares.
#evaluar si esta bueno que siempre se usen los mismos folds
kfold = StratifiedKFold(n_splits=5)
kfold.get_n_splits(X_desarrollo,y_desarrollo)


### K-fold CV 

In [ ]:
def primerTablaEjercicio2(treeClasifier):
    
    accuracy_train=[]
    accuracy_valildation=[]
    ROC_AUC_train=[]
    ROC_AUC_validation=[]

    #este for itera sobre los k folds en cada loop tego un set de datos y otro de validacion
    for train, test  in kfold.split(X_desarrollo,y_desarrollo):
        #print("TRAIN:", train_index,'\n', "TEST:", test_index,'\n' )
        X_train, X_val = X_desarrollo.iloc[train], X_desarrollo.iloc[test]
        y_train, y_val = y_desarrollo.iloc[train], y_desarrollo.iloc[test]
        #intancio el arbol que voy a entrenar en cada fold
        tree = treeClasifier(max_depth=3, criterion="gini")
        
        #from IPython.core.debugger import Tracer; Tracer()()

        tree.fit(X_train, y_train.astype(int))
        accuracy_train.append(tree.score(X=X_train, y=y_train))
        accuracy_valildation.append(tree.score(X=X_val, y=y_val))
        ROC_AUC_train.append(roc_auc_score(y_train,tree.predict(X_train)))
        ROC_AUC_validation.append(roc_auc_score(y_val,tree.predict(X_val)))
        
    return pd.DataFrame({ 'Partición' : np.arange(1,6),'Accuracy (training)' :accuracy_train,
                          'Accuracy (validación)' : accuracy_valildation,
                          'ROC AUC (training)' : ROC_AUC_train,
                          'ROC AUC (validación)' : ROC_AUC_validation})






# Tabla de precision

In [ ]:
display(primerTablaEjercicio2(DecisionTreeClassifier))  

# Arboles combinaciones

In [ ]:
def accuracyForTrainingAndValidation(depth,criteria,treeClasifier):
    acc_train=[]
    acc_val=[]


    #este for itera sobre los k folds en cada loop tego un set de training y otro de validacion
    for train, test  in kfold.split(X_desarrollo,y_desarrollo):
        #print("TRAIN:", train_index,'\n', "TEST:", test_index,'\n' )
        X_train, X_val = X_desarrollo.iloc[train], X_desarrollo.iloc[test]
        y_train, y_val = y_desarrollo.iloc[train], y_desarrollo.iloc[test]
        #intancio el arbol que voy a entrenar en cada fold
        tree = treeClasifier(max_depth=depth, criterion=criteria)

        tree.fit(X_train, y_train.astype(int))
        acc_train.append(tree.score(X=X_train, y=y_train))
        acc_val.append(tree.score(X=X_val, y=y_val))
        
    return {"training":np.mean(acc_train), "validation": np.mean(acc_val)}

        

In [ ]:
def segundaTablaEjercicio2(treeClasifier):

    trainingResults=[]
    validationResults=[]
    evaluatedDepths=[]
    evaluatedCriterias=[]
    depthsDictionary={3:'3',5:'5',None:'Infinito'}
    depths = [3,5,None]    
    criterias =['gini','entropy']

    for depth in depths:
        for criteria in criterias:
            trainingResults.append(accuracyForTrainingAndValidation(depth,criteria,treeClasifier)["training"])
            validationResults.append(accuracyForTrainingAndValidation(depth,criteria,treeClasifier)["validation"])
            evaluatedDepths.append(depthsDictionary[depth])
            evaluatedCriterias.append(criteria)
    
    return pd.DataFrame({ 'Altura Máxima' : evaluatedDepths,
                          'Criterio de evaluación de corte' : evaluatedCriterias,
                          'Accuracy (training)' : trainingResults,
                          'Accuracy (validación)' : validationResults}).sort_values(by=['Criterio de evaluación de corte'],ascending=False)

# Tabla con combinaciones

In [ ]:
display(segundaTablaEjercicio2(DecisionTreeClassifier))  

# Ejercicio Extra: Resultados para nuestro clasificador

In [ ]:
#display(primerTablaEjercicio2(ourTree.MiClasificadorArbol))  

In [ ]:
#display(segundaTablaEjercicio2(ourTree.MiClasificadorArbol))
#esta tabla tardo 10 minutos en crearse, ver como mejorar performance

In [3]:
roc_auc_score = make_scorer(roc_auc_score)
n_iter_search = 70
def performGridSearch(clasiffier, param_grid):
    start = time()
    gridSearch = GridSearchCV(clasiffier, param_grid, cv=kfold, scoring=roc_auc_score, return_train_score=False)
    gridSearch.fit(X_desarrollo, y_desarrollo)
    return {"bestScore" : gridSearch.best_score_, "bestParams" : gridSearch.best_params_, "executionTime" : time() - start, "allScores" : gridSearch.cv_results_}

def performRandomSearch(classifier, param_dist):
    start = time()
    randomSearch = RandomizedSearchCV(classifier, param_distributions=param_dist, n_iter=n_iter_search, cv=kfold, scoring=roc_auc_score, refit=True)
    randomSearch.fit(X_desarrollo, y_desarrollo)
    return {"bestScore" : randomSearch.best_score_, "bestParams" : randomSearch.best_params_, "executionTime" : time() - start}


def displayBestParamsTable(classifier, paramsGridSearch, paramsRandomSearch):
    gridSearchResult = performGridSearch(classifier, paramsGridSearch);
    randomSearchResult = performRandomSearch(classifier, paramsRandomSearch);
    df =  pd.DataFrame({ ' ' : ["Mejor performance (Roc Auc)"],
                          'Grid Search' : [gridSearchResult["bestScore"]],
                          'Random Search' : [randomSearchResult["bestScore"]]
                        })      
    display(df.set_index(' '))
    orderedBestGridParams = collections.OrderedDict(sorted(gridSearchResult["bestParams"].items()))
    orderedBestRandomParams = collections.OrderedDict(sorted(randomSearchResult["bestParams"].items()))
    display(pd.DataFrame({ 'Mejores parámetros Grid Search' : list(orderedBestGridParams.keys()),
                          '' : list(orderedBestGridParams.values())
                        }).set_index('Mejores parámetros Grid Search'))
    display(pd.DataFrame({ 'Mejores parámetros Random Search' : list(orderedBestRandomParams.keys()),
                          '' : list(orderedBestRandomParams.values())
                        }).set_index('Mejores parámetros Random Search'))
    
    allScores = renameAndDeleteColumnsInAllScoresTable(gridSearchResult["allScores"])
    
    display(HTML('<h6 style="font-style:/"/";font-size:/"88%/"">Todos los resultados del grid search (ordenados por performance):</h6>'))
    
    display(pd.DataFrame(allScores).sort_values(by=['']).set_index(""))
    
def renameAndDeleteColumnsInAllScoresTable(allScores):
    allScores.pop('mean_fit_time', None)
    allScores.pop('std_fit_time', None)
    allScores.pop('std_score_time', None)
    allScores.pop('params', None)
    allScores.pop('split0_test_score', None)
    allScores.pop('split1_test_score', None)
    allScores.pop('split2_test_score', None)
    allScores.pop('split3_test_score', None)
    allScores.pop('split4_test_score', None)
    allScores.pop('mean_score_time', None)
    allScores.pop('std_test_score', None)
    newDict = {}
    for key in list(allScores.keys()):
        if key.startswith("param_"):
            newDict[key[6:]] = allScores.pop(key);
    
    for key in list(newDict.keys()):
        allScores[key]=newDict[key]
    
    allScores['Roc Auc'] = allScores.pop("mean_test_score");
    allScores[''] = allScores.pop("rank_test_score");
    
    
    return allScores
    

# LDA

In [ ]:
lda_param_grid = [{'solver': ['lsqr'], 'shrinkage': [0, 1]}]

lda_param_random = {'solver': ['lsqr'], 'shrinkage': np.arange(0, 1, 0.000001)}

displayBestParamsTable(LinearDiscriminantAnalysis(),lda_param_grid, lda_param_random)

El algoritmo de LDA intenta partir el espacio de X en dos, defininedo con un hiperplano cuáles son las instancias
que pertenecen a una clase u otra. Este algoritmo funciona bien cuando cada una de las clases tiene una distribución normal y además se encuentran bien separadas. En el caso donde X es una matriz con múltiples columnas, el algoritmo asume que cada columna sigue una distribucion gaussiana de una dimensión, con alguna correlacion entre cada par de columnas. Esta correlacion se refleja en una matriz de co-varianzas, la cual es utilizada por el algoritmo. La matriz de co-varianza se calcula en base a las muestras, pero en la realidad es desconocida.
Skelarn provee distintos hiper parámetros para LDA. Nostros exploramos sobre Shrinkage y el solver lo dejamos fijo en 'lsqr'.

Solver: Según la documentación el solver por default es "svd", el cual no utiliza la matriz de covarianza, pero dado que la documentación no explica qué utiliza en vez, descartamos al solver "svd", teniendo en cuenta que además el algoritmo de LDA, tal cual se lo menciona en la bibliografía de la materia, utiliza una matriz de covarianza.
Al setear el solver en 'lsqr', el valor de shrinkage es tenido en cuenta, el cual puede tomar un valor entre 0 y 1; donde 0 significa computar la matriz de co-varianza empiricamente (con los datos de la muestra), y donde 1 significa utilizar
en vez una matriz diagonal que contiene la varianza de cada feature. Los valores intermedios corresponden a diferentes convinaciones lineales de ambas matrices. Si se usa el valor 'auto' para este parametro el valor de shrinkage se determina analiticamente segun el lema de Ledoit-Wolf el cual representa un mejor estimador de la matriz de covarianza real cuando la cantidad de features es muy grande ("http://scikit-learn.org/stable/modules/lda_qda.html#lda-qda"). Desde este punto de vista, coincide con nuestros resultados, dado que nuestro mejor resultado fue para shrinkage en 1, teniendo en cuenta que la cantidad de features es 200, un número relativamente grande, considerando además que solo contamos con 500 muestras.
En cuanto al random search, variamos el valor de shrinkage entre 0 y 1  con un step de 0.000001. Estos valores significan alguna matriz "interemedia" entre la matriz de co-varianzas y la matriz diagonal de varianzas. No podemos explicar matemáticamente que significa esto pero el algoritmo de Random Search encontró el mejor resultado con shrinkage en 0.568393, obteniendo un resultado de Roc Auc en 0.763814, mejor que el obenido en Grid Search.
Como conclusión general, dado que la performance no pudo superar un Roc Auc de 0.76, podemos decir que el algoritmo no puede
encontrar un hiperplano que divida muy bien las dos clases. Es decir que podemos conjeturar que las features no siguen distribuciones normales con las clases bien separadas entre sí.



# Arbol de Decisión

In [ ]:
tree_param_grid = [{'max_depth': [3,5,10,15,20,40,None],'criterion': ('gini','entropy') }]

tree_param_random = {'max_depth':np.append(np.arange(1, 41), None),'criterion': ('gini','entropy') }

displayBestParamsTable(DecisionTreeClassifier(),tree_param_grid, tree_param_random)



Antes de comenzar a analizar los resultados arrojados para la performance del algoritmo de Arboles de Decisión es importante comentar que cada vez que corremos el script de esta notebook, el orden de los resultados  para cada combinación varía mucho de corrida en corrida. Esto es por el fuerte componente aleatorio del algoritmo, relacionado con el hecho de que ante dos o más pares de (atributo / valor) que midan igual en ganancia de informacion o en ganancia gini, el algoritmo se quedará con alguno de ellos al azar. Aclaradndo además que en ninguna se alcanzo una performance superior al 0.70, explicamos los resultados arrojados para esta corrida.
Notamos que en este caso el criterio de entropía dio siempre mejor que el de gini. De todas maneras los criterios son muy similares y se suele usar gini por el hecho de que es más rapido para computar dado que no tiene función logaritmo.
En este caso el mejor de entropía dio 0.69 mientras que el mejor de gini dio 0.67. En otras corridas gini dio mejor que entropía.
Con respecto a las alturas, podemos ver que la performance no fue muy buena para ninguna de ellas. Una razon podría ser que para alturas altas el algoritmo tiende al overfitting, es decir anda bien cuando se testea con los datos de training pero no con los de validación. Por el otro lado con las alturas bajas, el algoritmo no logra generalizar, pues tiene en cuenta demasiado pocos atributos y descarta muchos que a la hora de la validación le terminan jugando en contra.
En este caso la mejor performance fue para altura en 20, pero para otras corridas la mejor performance fue para 5, 15, etc.
Cabe destacar que este es un algoritmo goloso, en el sentido que puede estar tomando decisiones malas, desde el punto de vista que pueden ser buenas decisiones en el contexto puntual en el que se toma la decisión pero malas desde un contexto global. 
El random search en este caso encontró una performance levemente superior del 0.70.

# KNN

In [ ]:
KNN_param_grid = {'n_neighbors': [1,5, 30, 50, 100], 'p' : [1,2], "weights" : ['uniform', 'distance']}
KNN_param_random = {'n_neighbors': np.arange(1, 101), 'p' : [1,2], "weights" : ['uniform', 'distance']}
displayBestParamsTable(KNeighborsClassifier(),KNN_param_grid, KNN_param_random)

Este algoritmo se basa en predecir la clase de una nueva instancia en base a las clases de sus "vecinos". Para esto, el algoritmo provee hiperparámetros como la cantidad de vecinos a tomar en cuenta, o "weights" que al setearlo en "distance" le da mayor importancia a los vecinos más cercanos y menor a los más lejanos, mientras que al setearlo en "uniform" le da igual importancia a todos. El parámetro "p" define el tipo de distancia, donde p define la norma p a ser tenida en cuenta. Nosotros exploramos con p en 1 y 2.
Para el valor de n buscamos en 1, 5, 30 50 y 100. Nuestro mejor resultado fue con n en 30, p en 2 y weights en "distance".
Notar que el hiperparámetro que mayor influyó en los resultados fue el del n, con resultados entre 0.72 y 0.75 para enes en 30, 50 y 100, y con los peores resultados para enes chicos; de 0.67 para n en 5, y en 0.60-0.61 para n en 1.
Notar que para un mismo n, variar el p o variar el weight no proporcionó mayores diferencias. La mayor diferencia fue de 0.21 parea el caso 11 en la tabla, que con un n en 30 dio un resultado de 0.729 que comparado al mejor caso, con n también en 30, dio 0.21 menos de Roc Auc. 
Podríamos pensar que para enes chicos el algoritmo funciona mal porque no logra generalizar y se pega demasiado a los valores del set de training. Es decir tiende al overfitting. Para distancias grandes funciona mejor, logrando una mejor generalización, aunque tampoco puede llegar a demasiados buenos resultados. Posiblemente no existe un ene fijo que pueda generalizar del todo bien para este set de datos, es decir, dependiendo de la instancia es posible que la cantidad de vecinos que mejor define la clase de la nueva instancia sea un ene diferente. Evidentemente para muchos casos, sucedió que la clase de la instancia nueva no era la de la mayoría de sus ene vecinos más cercanos, sino la contaria.
En esta corrida, no logramos obtener mejores resultados con el Random Search, obteniendo una performance de 0.74722, con n en 61, p en 2 y weights en distance.

# SVM

In [ ]:
SVM_param_grid = [{'kernel': ['rbf'], 'gamma': [1000,100,10,1,1e-1,1e-2,1e-3, 1e-4, 1e-5,1e-6,1e-7,1e-8,1e-9],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100,1000]}]
SVM_param_random = {'kernel': ['rbf'], 'gamma':sp.stats.expon(scale=.1),
                     'C': sp.stats.expon(scale=100)}
displayBestParamsTable(SVC(),SVM_param_grid, SVM_param_random)

El clasificador SVM, en el modo lineal, intenta separar las instancias en el espacio de X por medio de un hiperplano. En el modo no lineal, el algoritmo proyecta nuevas dimensiones como función de las dimensiones existentes, e intenta generar un hiperplano en este nuevo espacio.
El nombre del clasificador Support Vector Machines, hace referencia al hecho de que el algoritmo se basa unos vectores soporte que definen un margen que separará las dos clases. Para encontrar estos vectores, el algoritmo necesita de unas pocas instancias, las más cercanas a estos posibles vectores, mientras que el resto de las instancias no influyen en los mismos.
De esta manera intenta separar las clases, con un margen en el medio, siendo este margen más ancho o más angosto dependiendo del parámetro C. Este parámetro define de modo inverso cuanto error el algoritmo está dispuesto a aceptar, en el sentido de cuantas instancias pueden caer dentro del margen. Si el valor de C es grande, esto quiere decir que aceptamos poco error, y que entonces el margen es chico, de manera que pocas instancias caigan ahi dentro. De manera contraria si el valor de C es chico entonces toleramos más error.
Cuando resulta imposible encontrar un margen en el espacio de X que separe bien las instancias, se puede utilizar otro tipo de kernel, no lineal, que de manera implícita simula un espacio de mayores dimensiones. En nuestro caso exploramos con el kernel rbf, además del lineal.
Cuando utilizamos el kernel rbf, este se puede combinar con un nuevo hiperparámetro gamma. El gamma define de manera intuitiva cuán lejos alcanza la influencia de una instancia de training particular, con valores pequeños significando un alcance lejano y con valores grandes significando que el alcance es corto.
Si el gamma es demasiado grande, cada instancia de trainign define por si misma, junto con una pequeña distancia alrededor, que se area corresponde a la clase de ese punto, sería algo similar a definir KNN con cantidad de vecinos en uno, con altos riesgos de overfitting. En cambio, si el gamma es demasido chico, el modelo no podrá capturar la forma de los datos e intentará hacer una separación similar a la lineal.
En nuestro Grid Search exploramos para el kernel de rbf los gammas en [1000,100,10,1,1e-1,1e-2,1e-3, 1e-4, 1e-5,1e-6,1e-7,1e-8,1e-9]. Mientras que el C lo variamos en 1, 10, 100 y 1000.
Vemos en nuestro caso que todas las pruebas con gamma mayores o iguales que 1e-1 dieron una performance de 0.5, al igual que todos los resultados para gamams menores o iguales que 1e-8. De esta forma, los mejores gammas se encontraron en un rango intermedio entre 1e-2 y 1e-6, siendo la mejor peformance de 0.75 para gamam en 1e-05 y C en 100.
Notar que en el caso del Kernel linal, el mejor resultado fue de 0.68, esto indica que el algoritmo, restringido a las dimensiones de X, no pudo encontrar una buena forma de separar las clases.


# Descripcion

### Sepresentan en las tablas los mejores resultados segun score usando la metrica ROC_AUC  para cada clasificador y la correspondiente combinacion de hiperparametros. 

### para todos los clasificadores el score obtenido estuvo entre $0.7$ y $0.76$, una conjetura inicial fue la falta de entrenamiento para lo que ensayamos mas folds, para que el entrenamiento parcial a la hora de sacar un score contenga un conjunto de datos mayor, esto no cambio significativamente los valores. Luego se penso es el preproceso de datos y se probo con la normalizacion y escalamiento de los valores de las instancias, esto nuevamente no condujo a mejoras en las predicciones. Por último, dada la coincidencia de score para diferentes clasificadores , se penso en que el factor limitante sea la falta de datos o mas probablemente la eleccion de atributos.


